In [ ]:
# Imports
import numpy as np
from cv2 import imread, imwrite
# import pandas as pd
import matplotlib.pyplot as plt
from typing import Sequence

from skimage.morphology import area_opening, disk
from skimage.filters import median, gaussian

from PSFtools import *

from time import perf_counter
from Bindings_RLD import RLD_cpp, RLDf_cpp
from Bindings_SART import SART_cpp, SARTf_cpp
from Bindings_SMART import SMART_cpp, SMARTf_cpp

from Bindings_ArrayChecks import *

In [ ]:
dtype_choice = {"single":np.float32, "double":np.float64}

rld_choice = {"single": RLDf_cpp, "double": RLD_cpp}
sart_choice = {"single": SARTf_cpp, "double": SART_cpp}
smart_choice = {"single": SARTf_cpp, "double": SART_cpp}


# Reconstruction Settings
## Precision

In [ ]:
Precision = "single" # either 'single' or 'double'

DataType = dtype_choice[Precision]
RLD = rld_choice[Precision]

In [ ]:
z_cal_step = 20 # [um] between PSF images
z0 = 0 # PSF# representing z = 0

In [ ]:
def zToIndex(z):
    return int(z0 + z/z_cal_step)

## Image and Range

In [ ]:
# If the central PSF is not centered use these offsets
# They are the offset between the center of the image and the center of the central view

VerticalOffset = -50 # [pixels], Positive is up ^
HorizontalOffset = 50 # [pixels], Positive is right ->

# Assuming Circular Elemental View
# This is the radius of the elemental image in pixels
EI_radius = 220 # [pixels]

# Depth Range
z_min = -2000 # [um]
z_max = 500 # [um]

# Step Size
z_step = 20 # [um]

# Number of Iters
ITERS = 50

In [ ]:
FLAG = "LABEL"

In [ ]:
# z_min = -2000
# z_max = 300
# z_step = 40

zPlanes = np.arange(z_min, z_max+z_step/2, z_step)

zPlanes

# Load PSF Stack

## Specify PSF Filepaths

In [ ]:
# Generate PSF file paths
PSF_filepaths = [f"PATH_TO_PSF_FILES/{z:.0f}.tif" for z in zPlanes]

In [ ]:
# ReadPSF Stack, all PSFs put in a single 3D array
# Optional Argument 'clip' sets any intensity below (clip*max_value) to zero
# Found to be important for experimental images with noise
PSFstack, (L, M, N) = ReadPSFstack(PSF_filepaths, dtype=DataType, shift=True, clip=0.05)
# if the PSFs are centered (They should be) the shift Parameter performs an FFT shift on them
# they become ucentered so that the reconstruction is

# L = NUMBER OF Z PLANES
# M = NUMBER OF ROWS
# N = NUMBER OF COLUMNS

print(f"Stack Shape: ({L} x {M} x {N})")

# Import Raw Images for Reconstruction

In [ ]:
RawImageFileName = "PATH_TO_IMAGE_TO_RECONSTRUCT"
IMG = ReadRawImage(RawImageFileName, DataType, invert=True)

# Create Blank Volume

In [ ]:
def FillVol(L, M, N, EmptyVol, Radius, OffsetH = 0, OffsetV=0):

    # EmptyVol = np.zeros((self.nz, self.ny, self.nx), dtype=dtype)
    EmptyVol[:, :, :] = 1.0

    xpx = np.arange(0, N) - N//2 - OffsetH
    ypx = np.arange(0, M) - M//2 + OffsetV

    Xpx, Ypx = np.meshgrid(xpx, ypx)
    
    # Masks the space to a region around the center

    # ValidSpace = (Xpx**2 + Ypx**2) < (Radius**2) # Circular region
    ValidSpace = (np.abs(Xpx) < Radius) * (np.abs(Ypx) < Radius) # Square region
    # ValidSpace = (np.abs(Xpx) < 3/2*Radius)*(np.abs(Ypx) < Radius) # Rectangular region

    for i in range(L):
        EmptyVol[i, :, :] *= (ValidSpace) #.astype(dtype))

In [ ]:
VOL = np_empty_16byteAligned((L, M, N), DataType)
VOL[:, :, :] = 0.0

FillVol(L, M, N, VOL, EI_radius, OffsetH = HorizontalOffset, OffsetV = VerticalOffset)
ROI_j = (M//2 - EI_radius - VerticalOffset, M//2 + EI_radius + 1 - VerticalOffset)
ROI_i = (N//2 - EI_radius + HorizontalOffset, N//2 + EI_radius + 1 + HorizontalOffset)

## Create Image Mask using PSFs

Found to be important for numerical stability to mask image regions outside of elemental images

Can also manually specify a mask to use
If one has been made previously it will use that one

In [ ]:
from RLD import ProjF_noFT

def CreateMask(EmptyVol, PSF):

    FP = ProjF_noFT(EmptyVol, PSF)

    return FP / L

In [ ]:
Mask = None

if 'Mask' in locals():
    Mask = CreateMask(VOL, PSFstack)
    Mask = Mask > (2**-15) #0.01*Mask.max()
    imwrite(f"{'.'.join(PSF_filepaths[0].split('.')[:-1])}_MASK_{FLAG}.png", Mask.astype(np.uint8))
elif Mask.split('.')[-1] == ".npy":
    Mask = np.load(Mask)
else:
    Mask = ReadRawImage(Mask, DataType)

In [ ]:
# show the generated mask

fig, ax = plt.subplots()
ax.set_title("Mask")
ax.imshow(Mask)

## Apply Mask to the Image
This is for numerical stability stuff

In [ ]:
IMG *= Mask

# Reconstruct

In [ ]:
# t0 = perf_counter()
# RLD(VOL, PSFstack, IMG, ITERS)
# tf = perf_counter()
# print(f"RLD w/ C++ Time: {tf-t0} sec")

In [ ]:
# If any one of these conditions are met the RLD output is invalid
# Try reurunning all the cells from PSFstack creation down
# When a new volume size is run for the first time it usually fails
# I sort of know why this is, but in a more real way I don't

if VOL.min() < -1e-16:
    raise ValueError(f"RLD output is negative, minimum = {VOL.min()} at {VOL.argmin()}")
if VOL.max() == 0:
    if PSFstack.max() == 0:
        raise ValueError("RLD output and PSF are all zero")
    raise ValueError("RLD output all zeros")
if np.isnan(VOL).any():
    raise ValueError("RLD output has NaNs")

## Reprojection vs Original Image

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 24), dpi=350)


vmin, vmax = IMG.min(), IMG.max()

ax[0].set_title("original")
ax[0].imshow(IMG)


FP = ProjF_noFT(VOL, PSFstack)
ax[1].set_title("Reconstruction Projection")
ax[1].imshow(FP)

ax[2].set_title("Difference")
cb = ax[2].imshow(np.log2(IMG/FP), cmap='Spectral')
fig.colorbar(cb, ax=ax[2])

# Export Volumes as Arrays

In [ ]:
VOL_crop = VOL[:, ROI_j[0]:ROI_j[1], ROI_i[0]:ROI_i[1]]

## As Image Stack 
(Intensities Adjusted)

In [ ]:
from cv2 import imwrite
from os import mkdir

from datetime import datetime
now = datetime.now()
RunTime = now.strftime("%Y_%m_%d_%H.%M.%S")

In [ ]:
bit_depth = 16

# Normalize Intensity to make use of the bit depth
VOL_crop /= VOL_crop.max()
VOL_crop *= (2**(16) - 1)

RawImgPathFolder = f"{'.'.join(RawImageFileName.split('.')[:-1])}"

try: mkdir(RawImgPathFolder)
except FileExistsError: pass

try: mkdir(f"{RawImgPathFolder}/zStack_{RunTime}_{FLAG}")
except FileExistsError: pass

for k in range(L):
    Slice = VOL_crop[k, :, :].astype(np.uint16)

    zOffset = zPlanes[0]

    imwrite(f"{RawImgPathFolder}/zStack_{RunTime}_{FLAG}/z{k:03d}.tif", Slice)

np.savez(f"{RawImgPathFolder}/log.npz", Z=zPlanes, VerticalOffset=VerticalOffset, HorizontalOffset=HorizontalOffset,
         z_min = z_min, z_max = z_max, z_step=z_step, ITERS = ITERS, EI_radius=EI_radius)

## As compressed numpy Array

In [ ]:
# np.savez_compressed(f"{RawImgPathFolder}/3D_RLD.npz", Volume=VOL_crop)